<h3>Import các thư viện</h3>

In [1]:
from preprocess import Preprocessor # import các hàng tiền xử lí dữ liệu

#thư viện dùng cho đọc file
import pandas as pd # type: ignore

#thư viện làm việc với mảng
import numpy as np # type: ignore

#import model 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument # type: ignore
from gensim.utils import simple_preprocess # type: ignore
from gensim.models import Phrases # type: ignore
# Bỏ qua tất cả các cảnh báo
import warnings
warnings.filterwarnings("ignore")

#import logger
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



<h3>Đọc dữ liệu</h3>

In [2]:
#lấy đường dẫn
relative_path = "../../resources/data/data.csv"
#đọc file csv
sample_df = pd.read_csv(relative_path)
df = sample_df # tạo copy

#đọc thử 5 mẫu dữ liệu đầu tiên
df.head()

,Article link,Website source,Article type,Article title,Content,Creation date,Author,Category,Tags,Summary,Temp
0,https://www.theblock.co/post/285730/custodia-i...,https://www.theblock.co,News Article,Custodia is not entitled to a Fed master accou...,The Federal Reserve does not have to give digi...,"March 29, 2024, 7:05PM EDT",Sarah Wynn,Policy,COURT HEARINGS-LAWSUITS,Custodia Bank sued the central bank in 2022 fo...,NaN
1,https://www.theblock.co/post/285724/multicoin-...,https://www.theblock.co,News Article,"Multicoin Capital's hedge fund has grown 9,281...",Multicoin Capital’s crypto-focused hedge fund ...,"March 29, 2024, 7:00PM EDT UPDATED: March 29, ...",Elizabeth Napolitano,Companies,INVESTMENT FIRMS,"Multicoin Capital’s hedge fund has returned 9,...",NaN
2,https://www.theblock.co/post/285702/1kx-raise-...,https://www.theblock.co,News Article,1kx raises $75 million in latest funding round,"1kx has raised $75 million, the latest sign in...","March 29, 2024, 3:20PM EDT",Elizabeth Napolitano,Companies,NaN,Investment firm 1kx has raised $75 million for...,NaN
3,https://www.theblock.co/post/285690/cftc-commi...,https://www.theblock.co,News Article,CFTC Commissioner Pham says agency may be infr...,One of the Commodity Futures Trading Commissio...,"March 29, 2024, 12:06PM EDT",Sarah Wynn,Exchanges,CFTC-SEC,The agency’s complaint “appears to assert that...,NaN
4,https://www.theblock.co/post/285608/bitcoin-fu...,https://www.theblock.co,News Article,Bitcoin futures open interest reaches new high...,Open interest for bitcoin futures on centraliz...,"March 29, 2024, 11:03AM EDT UPDATED: March 29,...",Vishal Chawla,The Block,NaN,Bitcoin futures open interest on centralized e...,NaN


- <h5>Khảo sát định dạng các dữ liệu</h5>

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8334 entries, 0 to 8333
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Article link     8334 non-null   object 
 1    Website source  8334 non-null   object 
 2    Article type    8334 non-null   object 
 3    Article title   8334 non-null   object 
 4    Content         8331 non-null   object 
 5    Creation date   8334 non-null   object 
 6    Author          8333 non-null   object 
 7    Category        8334 non-null   object 
 8    Tags            5908 non-null   object 
 9    Summary         8129 non-null   object 
 10   Temp            0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 716.3+ KB


<h3>Tiền xử lý dữ liệu</h3>
<ul><li><h5>Tiền xử lý các trường text</h5></li><ul>

In [4]:
preprocessor = Preprocessor()

df[' Article title'] = df[' Article title'].apply(preprocessor.preprocess_text)
df[' Author'] = df[' Author'].apply(preprocessor.preprocess_text)
df[' Content'] = df[' Content'].apply(preprocessor.preprocess_text)
df[' Category'] = df[' Category'].apply(preprocessor.preprocess_text)

df.head(10)#đọc thử dữ liệu


,Article link,Website source,Article type,Article title,Content,Creation date,Author,Category,Tags,Summary,Temp
0,https://www.theblock.co/post/285730/custodia-i...,https://www.theblock.co,News Article,custodia entitle feed master account judge rule,federal reserve digital asset bank custodia ac...,"March 29, 2024, 7:05PM EDT",sarah wynn,policy,COURT HEARINGS-LAWSUITS,Custodia Bank sued the central bank in 2022 fo...,NaN
1,https://www.theblock.co/post/285724/multicoin-...,https://www.theblock.co,News Article,multicoin capital hedge fund grow 9281 2017 in...,multicoin capital cryptofocused hedge fund bri...,"March 29, 2024, 7:00PM EDT UPDATED: March 29, ...",elizabeth napolitano,company,INVESTMENT FIRMS,"Multicoin Capital’s hedge fund has returned 9,...",NaN
2,https://www.theblock.co/post/285702/1kx-raise-...,https://www.theblock.co,News Article,1 kx raise 75 million fund round,1 kx raise 75 million sign investor embrace cr...,"March 29, 2024, 3:20PM EDT",elizabeth napolitano,company,NaN,Investment firm 1kx has raised $75 million for...,NaN
3,https://www.theblock.co/post/285690/cftc-commi...,https://www.theblock.co,News Article,cftc commissioner pham agency infringe sec aut...,commodity future trade commission leader carol...,"March 29, 2024, 12:06PM EDT",sarah wynn,exchange,CFTC-SEC,The agency’s complaint “appears to assert that...,NaN
4,https://www.theblock.co/post/285608/bitcoin-fu...,https://www.theblock.co,News Article,bitcoin future reach 38 billion,bitcoin future centralize exchange reach fresh...,"March 29, 2024, 11:03AM EDT UPDATED: March 29,...",vishal chawla,block,NaN,Bitcoin futures open interest on centralized e...,NaN
5,https://www.theblock.co/post/285659/grayscale-...,https://www.theblock.co,News Article,grayscale launch proofofstake investment fund ...,grayscale announce launch dynamic income fund ...,"March 29, 2024, 10:57AM EDT",rt watson,fund,NaN,Grayscale is launching a “dynamic income fund”...,NaN
6,https://www.theblock.co/post/285655/uk-judge-f...,https://www.theblock.co,News Article,uk court freeze 76 million worth craig wright ...,uk judge approve freeze 6 million 76 million w...,"March 29, 2024, 10:46AM EDT",mk manoylov,policy,BITCOIN-COURT HEARINGS,A UK judge approved a so-called ‘worldwide fre...,NaN
7,https://www.theblock.co/post/285590/prismafi-h...,https://www.theblock.co,News Article,prisma hackerlinked wallet move fund tornado c...,wallet connect hacker prisma finance 116 milli...,"March 29, 2024, 6:13AM EDT UPDATED: March 29, ...",danny park,security,EXPLOITS-HACKS,Prisma Finance was hacked for over $11.6 milli...,NaN
8,https://www.theblock.co/post/285587/ethereum-v...,https://www.theblock.co,News Article,ethereum cofounder vitalik buterin memecoins b...,blog post publish ethereum cofounder vitalik b...,"March 29, 2024, 5:37AM EDT",adam james,crypto ecosystem,VITALIK BUTERIN,Vitalik Buterin published a new blog post on t...,NaN
9,https://www.theblock.co/post/285564/spot-bitco...,https://www.theblock.co,News Article,spot bitcoin etfs 4 consecutive day net inflow...,flow spot bitcoin exchangetraded fund remain n...,"March 29, 2024, 4:03AM EDT",adam james,fund,NaN,The daily total net inflow for the spot bitcoi...,NaN


- <h5>Tiền xử lý date</h5>

In [5]:
df[' Creation date'] = df[' Creation date'].apply(preprocessor.preprocess_date)

df.head(10)

,Article link,Website source,Article type,Article title,Content,Creation date,Author,Category,Tags,Summary,Temp
0,https://www.theblock.co/post/285730/custodia-i...,https://www.theblock.co,News Article,custodia entitle feed master account judge rule,federal reserve digital asset bank custodia ac...,2024-03-29,sarah wynn,policy,COURT HEARINGS-LAWSUITS,Custodia Bank sued the central bank in 2022 fo...,NaN
1,https://www.theblock.co/post/285724/multicoin-...,https://www.theblock.co,News Article,multicoin capital hedge fund grow 9281 2017 in...,multicoin capital cryptofocused hedge fund bri...,2024-03-29,elizabeth napolitano,company,INVESTMENT FIRMS,"Multicoin Capital’s hedge fund has returned 9,...",NaN
2,https://www.theblock.co/post/285702/1kx-raise-...,https://www.theblock.co,News Article,1 kx raise 75 million fund round,1 kx raise 75 million sign investor embrace cr...,2024-03-29,elizabeth napolitano,company,NaN,Investment firm 1kx has raised $75 million for...,NaN
3,https://www.theblock.co/post/285690/cftc-commi...,https://www.theblock.co,News Article,cftc commissioner pham agency infringe sec aut...,commodity future trade commission leader carol...,2024-03-29,sarah wynn,exchange,CFTC-SEC,The agency’s complaint “appears to assert that...,NaN
4,https://www.theblock.co/post/285608/bitcoin-fu...,https://www.theblock.co,News Article,bitcoin future reach 38 billion,bitcoin future centralize exchange reach fresh...,2024-03-29,vishal chawla,block,NaN,Bitcoin futures open interest on centralized e...,NaN
5,https://www.theblock.co/post/285659/grayscale-...,https://www.theblock.co,News Article,grayscale launch proofofstake investment fund ...,grayscale announce launch dynamic income fund ...,2024-03-29,rt watson,fund,NaN,Grayscale is launching a “dynamic income fund”...,NaN
6,https://www.theblock.co/post/285655/uk-judge-f...,https://www.theblock.co,News Article,uk court freeze 76 million worth craig wright ...,uk judge approve freeze 6 million 76 million w...,2024-03-29,mk manoylov,policy,BITCOIN-COURT HEARINGS,A UK judge approved a so-called ‘worldwide fre...,NaN
7,https://www.theblock.co/post/285590/prismafi-h...,https://www.theblock.co,News Article,prisma hackerlinked wallet move fund tornado c...,wallet connect hacker prisma finance 116 milli...,2024-03-29,danny park,security,EXPLOITS-HACKS,Prisma Finance was hacked for over $11.6 milli...,NaN
8,https://www.theblock.co/post/285587/ethereum-v...,https://www.theblock.co,News Article,ethereum cofounder vitalik buterin memecoins b...,blog post publish ethereum cofounder vitalik b...,2024-03-29,adam james,crypto ecosystem,VITALIK BUTERIN,Vitalik Buterin published a new blog post on t...,NaN
9,https://www.theblock.co/post/285564/spot-bitco...,https://www.theblock.co,News Article,spot bitcoin etfs 4 consecutive day net inflow...,flow spot bitcoin exchangetraded fund remain n...,2024-03-29,adam james,fund,NaN,The daily total net inflow for the spot bitcoi...,NaN


<h3>Đánh trọng số cho các cột</h3>

In [6]:
column_weights = {
    ' Author': 1,
    ' Content': 1,
    ' Category': 1,
    ' Article title': 5
}

<h3>Các hàm tokenize và tạo TaggedDocument</h3>

In [7]:
def tokenize_cell(cell): #tokenize 1 cell
    # Tokenize một từ bằng cách sử dụng Gensim
    return simple_preprocess(cell)

def tokenize_row(row, column_weights):
    sentences = []
    for column, weight in column_weights.items():
        if column in row:
            cell_tokens = tokenize_cell(row[column])
            sentences.extend([cell_tokens] * weight)
    return sentences



In [8]:
all_sentences = []
for index, row in df.iterrows():
    sentences = tokenize_row(row, column_weights)
    all_sentences.extend(sentences)

# Chuyển mảng all_sentences thành một NumPy array
print(len(all_sentences))

66672


In [9]:
bigram_model = Phrases(sentences, min_count=1, threshold=1)

2024-04-22 11:20:06,503 : INFO : collecting all words and their counts
2024-04-22 11:20:06,504 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-04-22 11:20:06,505 : INFO : collected 207 token types (unigram + bigrams) from a corpus of 149 words and 8 sentences
2024-04-22 11:20:06,506 : INFO : merged Phrases<207 vocab, min_count=1, threshold=1, max_vocab_size=40000000>
2024-04-22 11:20:06,507 : INFO : Phrases lifecycle event {'msg': 'built Phrases<207 vocab, min_count=1, threshold=1, max_vocab_size=40000000> in 0.00s', 'datetime': '2024-04-22T11:20:06.507919', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}


In [10]:
def read_corpus(data, column_weights, bigram_model=None, tokens_only=False):
    
    if bigram_model is not None:
        for index, row in data.iterrows():
            tokens = []
            sentences = tokenize_row(row, column_weights)
            for sentence in sentences:
                if bigram_model:
                    sentence = bigram_model[sentence]
                tokens.extend(sentence)
            if tokens_only:
                yield tokens
            else:
                yield TaggedDocument(tokens, [index])
    else:
        for index, row in data.iterrows():
            sentences = tokenize_row(row, column_weights)
            if tokens_only:
                for sentence in sentences:
                    yield sentence
            else:
                for sentence in sentences:
                    yield TaggedDocument(sentence, [index])

<h3>Tạo TaggedDocuments và đọc thử</h3>

In [11]:
tagged_docs = list(read_corpus(data=df, column_weights=column_weights, bigram_model=bigram_model))
print(len(tagged_docs))
tagged_docs[:2]


8334


[TaggedDocument(words=['sarah', 'wynn', 'federal', 'reserve', 'digital', 'asset', 'bank', 'custodia', 'access', 'master', 'account', 'judge', 'rule', 'friday', 'federal', 'reserve', 'bank', 'discretion', 'grant', 'master', 'account', 'judge', 'scott', 'skavdahl', 'district', 'court', 'district', 'wyoming', 'judgment', 'news', 'blow', 'custodia', 'bank', 'sue', 'federal', 'reserve', 'board', 'governor', 'federal', 'reserve', 'bank', 'kansas', 'city', 'delay', 'decision', 'application', 'central', 'bank', 'master', 'account', 'master', 'account', 'allow', 'institution', 'direct', 'access', 'feds', 'payment', 'system', 'provide', 'direct', 'access', 'uss', 'money', 'supply', 'available', 'financial', 'institution', 'master', 'account', 'force', 'rely', 'partner', 'bank', 'master', 'account', 'provide', 'service', 'custodias', 'position', 'correct', 'effectively', 'mean', 'depository', 'institution', 'charter', 'law', 'regardless', 'soundly', 'craft', 'entitle', 'master', 'account', 'allow

<h3>Train model</h3>

In [12]:
model = Doc2Vec(vector_size=50, window=5, min_count=1, epochs=50)
model.build_vocab(tagged_docs)
model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)


2024-04-22 11:20:10,514 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,s0.001,t3>', 'datetime': '2024-04-22T11:20:10.514447', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2024-04-22 11:20:10,515 : INFO : collecting all words and their counts
2024-04-22 11:20:10,516 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2024-04-22 11:20:10,827 : INFO : collected 35308 word types and 8334 unique tags from a corpus of 8334 examples and 1879420 words
2024-04-22 11:20:10,827 : INFO : Creating a fresh vocabulary
2024-04-22 11:20:10,951 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 35308 unique words (100.00% of original 35308, drops 0)', 'datetime': '2024-04-22T11:20:10.951276', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37

<h3>Thử tạo doc2vec với model</h3>

In [24]:
vector = model.infer_vector(['sam'], epochs=100, alpha=0.025)
print(vector)
vector = model.infer_vector(['sam'], epochs=100, alpha=0.025)
print(vector)

[-0.08472066 -0.3814029  -0.13749507  0.14280608  0.18344519 -0.43797252
 -0.00131482  0.09062125 -0.75109094 -0.3166953   0.03734662 -0.0946426
 -0.05798193  0.12246471 -0.272047    0.33504972 -0.0440779  -0.25234354
 -0.40062043 -0.23572119  0.08717984  0.02420101  0.4348501   0.4634126
  0.05005372 -0.1613694   0.09647759 -0.44425708 -0.28749573  0.05585107
  0.13848215 -0.0833867  -0.02322724 -0.02537507 -0.24344708  0.19273558
 -0.16100723 -0.15402715  0.03080525  0.10347532  0.32036057 -0.07890413
 -0.38040835  0.21926567  0.06217869 -0.23046252 -0.49511516 -0.24134375
 -0.08882642 -0.01888806]
[-9.28080007e-02 -3.61808747e-01 -1.97012559e-01  1.69201896e-01
  2.20128059e-01 -3.65934700e-01 -7.43422518e-03  7.31468201e-02
 -7.50606239e-01 -2.55774438e-01 -3.07311416e-02 -6.73507750e-02
 -7.46945739e-02  1.79819539e-01 -2.74175197e-01  3.27339530e-01
 -8.18191990e-02 -2.94930726e-01 -3.96862477e-01 -2.31111541e-01
  5.75530790e-02  3.30463573e-02  3.60534132e-01  4.43160027e-01
  

<h3>Lưu model</h3>

In [14]:
model.save('models/doc2vec_model.model')

2024-04-22 11:21:34,937 : INFO : Doc2Vec lifecycle event {'fname_or_handle': 'models/doc2vec_model.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2024-04-22T11:21:34.937942', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'saving'}
2024-04-22 11:21:34,938 : INFO : not storing attribute cum_table
2024-04-22 11:21:34,969 : INFO : saved models/doc2vec_model.model
